In [38]:
import pandas as pd
import numpy as np
from keras.layers import Dense, Dropout, Activation, Input
from keras.models import Model

In [39]:
base = pd.read_csv('games.csv')
base.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


## Objetivo é determinar as vendas na américa do norte(NA_sales), Europa(EU_sales) e Japão(JP_sales). Se quisesse predizer as vendas totais era só usar conforme exemplo anterior de regressão com uma única saída para a variavél Global_Sales.

In [40]:
base = base.drop('Other_Sales', axis = 1)
base = base.drop('Global_Sales', axis = 1)
base = base.drop('Developer', axis = 1) # O campo publisher e bem parecido, e o Developer tem muitos 'nan's

## Apagando linhas que tem muitos dados faltantes, não é maio recomendavel, mas será usado aqui nesse exemplo, em outros casos deve ser analisado se vale a pena preencher com a média de tudo.

In [41]:
base = base.dropna(axis = 0)

In [42]:
# Apagar os dados, se deixava os valores menores que 1, valores decorrelação muito baixo e erro muito alto
# Com os dados do Japão não é necessario não entendi pq, perguntei lá no forum do curso

base = base.loc[base['NA_Sales'] > 1]
base = base.loc[base['EU_Sales'] > 1]


In [43]:
base['Name'].value_counts()
# desbalanceado, exclui do datasett

Grand Theft Auto V                        4
Assassin's Creed IV: Black Flag           3
Call of Duty: Ghosts                      3
The Elder Scrolls V: Skyrim               3
Battlefield 4                             3
                                         ..
Spider-Man: The Movie                     1
James Bond 007: Agent Under Fire          1
Forza Motorsport 4                        1
Tony Hawk's Pro Skater                    1
World of Warcraft: The Burning Crusade    1
Name: Name, Length: 223, dtype: int64

In [44]:
nome_jogos = base.Name # Salvando a feature Name em uma outra varíavel

venda_na_pd = base.NA_Sales
venda_eu_pd = base.EU_Sales
venda_jp_pd = base.JP_Sales

In [45]:
nome_jogos

0                      Wii Sports
2                  Mario Kart Wii
3               Wii Sports Resort
6           New Super Mario Bros.
7                        Wii Play
                  ...            
568                Medal of Honor
573    Tom Clancy's Splinter Cell
591        Mario Strikers Charged
610                    Crazy Taxi
636         The Sims: Bustin' Out
Name: Name, Length: 258, dtype: object

In [46]:
base = base.drop('Name', axis = 1)

## Dividir os dados previsores e as três variavei a serem preditas

In [64]:
previsores = base.iloc[:, [0,1,2,3,7,8,9,10,11]].values
venda_na = base.iloc[:, 4].values
venda_eu = base.iloc[:, 5].values
venda_jp = base.iloc[:, 6].values

In [48]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [49]:
labelencoder = LabelEncoder()

In [50]:
# Converter para variavel numerica, variaveis categoricas com strings : no caso 0,2,3,
previsores_pd = base.iloc[:, [0,1,2,3,7,8,9,10,11]]
previsores_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258 entries, 0 to 636
Data columns (total 9 columns):
Platform           258 non-null object
Year_of_Release    258 non-null float64
Genre              258 non-null object
Publisher          258 non-null object
Critic_Score       258 non-null float64
Critic_Count       258 non-null float64
User_Score         258 non-null object
User_Count         258 non-null float64
Rating             258 non-null object
dtypes: float64(4), object(5)
memory usage: 20.2+ KB


In [51]:
previsores[0]

array(['Wii', 2006.0, 'Sports', 'Nintendo', 76.0, 51.0, '8', 322.0, 'E'],
      dtype=object)

In [52]:
previsores[:,0] = labelencoder.fit_transform(previsores[:,0])
previsores[:,2] = labelencoder.fit_transform(previsores[:,2])
previsores[:,3] = labelencoder.fit_transform(previsores[:,3])
previsores[:,6] = labelencoder.fit_transform(previsores[:,6])
previsores[:,8] = labelencoder.fit_transform(previsores[:,8])

In [53]:
onehotencoder = OneHotEncoder(categorical_features = [0,2,3,6,8])

In [54]:
previsores = onehotencoder.fit_transform(previsores).toarray()

/home/jarbasjr/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/jarbasjr/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [55]:
previsores.shape

(258, 109)

# Estrutura da Rede Neural

## Vamos fazer um pouco diferente do que foi feito anteriormente no modelo "Sequential", onde nos iamos adicionando as camadas, como nós temos um problema um pouco diferente do que vimos anteriormente, vamos usar alguns outros recursos do keras para que vocêa aprenda a construir uma rede neural mais robusta para que você consiga ter um maior controle sobre as camadas. Não necessariamente você precisar usar aquele modelo Sequencial, aqui é uma outra maneira que irá agregar ao seu repertório de programação de utiulização do keras para construir as redes neurais.

In [56]:
camada_entrada = Input(shape =(114,)) ## 114 numero de entradas

#sigmoid, pois segundo o ministrante do curso em testes anteriores funcionaou melhor.
# units = (114+3) / 2 = 58,5, arrendoda para 59
camada_oculta1 = Dense(units = 59, activation = 'sigmoid')(camada_entrada) 
camada_oculta2 = Dense(units = 59, activation = 'sigmoid')(camada_oculta1)

# como são multiplas variaveis, três neuronios de saida
camada_saida1 = Dense(units = 1, activation = 'linear')(camada_oculta2)
camada_saida2 = Dense(units = 1, activation = 'linear')(camada_oculta2)
camada_saida3 = Dense(units = 1, activation = 'linear')(camada_oculta2)

In [ ]:
regressor = Model(inputs = camada_entrada,
                     outputs = [camada_saida1, camada_saida2, camada_saida3])

regressor.compile(optimizer = 'adam',loss = 'mse')

regressor.fit(previsores, [venda_na, venda_eu, venda_jp],
                 epochs = 500, batch_size = 100)

## É claro que aqui é um exemplo só para o entendimento de como funciona o processo, não estamos trabalhando com base de dados de treino e teste ou validação cruzada. Nós estamos treinando nessa base de dados e estamos também fazendo a avaliação nessa mesma base dedados, porém não é a abordagem correta, então é necessario usar validação cruzada com base em daods de treino e teste.

In [ ]:
previsao_na, previsao_eu, previsao_jp = regressor.predict(previsores)

In [57]:
nome_jogos.head()

0               Wii Sports
2           Mario Kart Wii
3        Wii Sports Resort
6    New Super Mario Bros.
7                 Wii Play
Name: Name, dtype: object

In [65]:
venda_na_pd.head()

0    41.36
2    15.68
3    15.61
6    11.28
7    13.96
Name: NA_Sales, dtype: float64

In [ ]:
previsao_na